# Calibrate Dashcam


https://docs.opencv.org/2.4/doc/tutorials/calib3d/camera_calibration/camera_calibration.html

https://medium.com/vacatronics/3-ways-to-calibrate-your-camera-using-opencv-and-python-395528a51615

https://towardsdatascience.com/computer-vision-for-lane-finding-24ea77f25209

Use a standard OpenCV process to correct for distortions caused by the optics of the camera

In [1]:
import os
import sys
import cv2
import numpy as np
import pathlib

from datetime import datetime

from matplotlib import pyplot as plt
%matplotlib inline

module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)

## Configuration

In [7]:
# Path to directory where calibration images are saved
images_dir    = os.path.join(module_path_root, 'data_sources', 'dashcam_calibration_images') # Path to calibration images
sample_dir    = os.path.join(module_path_root, 'data_sources', 'dashcam_samples')
images_format = 'png' # File format of the calibration images
square_size   = 2.52   # sice of squares on the test image in centimetres
width         = 9      # width of the pattern
height        = 6      # height of the pattern
config_file   = os.path.join(module_path_root, 'data_sources', 'dashcam_calibration.yml')

## Function Definitions

From: https://medium.com/vacatronics/3-ways-to-calibrate-your-camera-using-opencv-and-python-395528a51615
Adapted from C++ code at: https://docs.opencv.org/2.4/doc/tutorials/calib3d/camera_calibration/camera_calibration.html

In [3]:
def calibrate_chessboard(dir_path, image_format, square_size, width, height):
    '''Calibrate a camera using chessboard images.'''
    # termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,6,0)
    objp = np.zeros((height*width, 3), np.float32)
    objp[:, :2] = np.mgrid[0:width, 0:height].T.reshape(-1, 2)

    objp = objp * square_size

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    print(dir_path)
    images = pathlib.Path(dir_path).glob(f'*.{image_format}')
    # Iterate through all images
    
    for idx, fname in enumerate(images):
        print('{0:s} Processing image # {1:d} {2:s}'.format(str(datetime.now()), idx, str(fname)))
        
        img = cv2.imread(str(fname))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (width, height), None)

        # If found, add object points, image points (after refining them)
        if ret:
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)

    # Calibrate camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return [ret, mtx, dist, rvecs, tvecs]

def save_coefficients(mtx, dist, path):
    '''Save the camera matrix and the distortion coefficients to given path/file.'''
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_WRITE)
    cv_file.write('K', mtx)
    cv_file.write('D', dist)
    # note you *release* you don't close() a FileStorage object
    cv_file.release()

def load_coefficients(path):
    '''Loads camera matrix and distortion coefficients.'''
    # FILE_STORAGE_READ
    cv_file = cv2.FileStorage(path, cv2.FILE_STORAGE_READ)

    # note we also have to specify the type to retrieve other wise we only get a
    # FileNode object back instead of a matrix
    camera_matrix = cv_file.getNode('K').mat()
    dist_matrix = cv_file.getNode('D').mat()

    cv_file.release()
    return [camera_matrix, dist_matrix]

## Convert calibration video to images

In [4]:
def split_video(filename, output_dir, video_num, interval):
    cap = cv2.VideoCapture(filename)
    
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    frame_num = 0
    
    while True:
        ret, frame = cap.read()
        frame_num += 1
        
        if ret == True:
            if frame_num % interval == 0:
                filename = 'frame_{0:02d}_{1:05d}.png'.format(video_num, frame_num)
                print('Writing: {0:s}'.format(filename))
                cv2.imwrite(os.path.join(output_dir, filename), frame)
        else:
            break
    
    cap.release()       
        

In [ ]:
video1 = os.path.join(images_dir, 'distortion_calibration1.mp4')
video2 = os.path.join(images_dir, 'distortion_calibration2.mp4')

split_video(video1, images_dir, 1, 60)
split_video(video2, images_dir, 2, 60)

In [5]:
ret, mtx, dist, rvecs, tvecs = calibrate_chessboard(images_dir, images_format, square_size, width, height)

D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images
2021-09-23 12:13:24.482593 Processing image # 0 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_01_01140.png
2021-09-23 12:13:24.564927 Processing image # 1 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_01_01200.png
2021-09-23 12:13:43.467006 Processing image # 2 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_01_01260.png
2021-09-23 12:13:43.545842 Processing image # 3 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_01_01320.png
2021-09-23 12:13:43.611810 Processing image # 4 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_01_01380.png
2021-09-23 12:13:43.684226 Processing image # 5 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_01_01440.png
2021-09-23 12:13:43.754186 Processing image #

2021-09-23 12:17:14.553548 Processing image # 56 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_02_02520.png
2021-09-23 12:17:44.255494 Processing image # 57 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_02_02580.png
2021-09-23 12:18:13.731291 Processing image # 58 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_02_02640.png
2021-09-23 12:18:14.605217 Processing image # 59 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_02_02700.png
2021-09-23 12:18:43.208053 Processing image # 60 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_02_02760.png
2021-09-23 12:18:44.027916 Processing image # 61 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibration_images\frame_02_02820.png
2021-09-23 12:19:08.030331 Processing image # 62 D:\TensorFlow2\TFODCourse\minor_thesis\data_sources\dashcam_calibrati

In [6]:
save_coefficients(mtx, dist, config_file)

## Apply to a sample dashcam image

In [9]:
split_video(os.path.join(sample_dir, 'FILE210920-145008F.MP4'), sample_dir, 1, 60)

Writing: frame_01_00060.png
Writing: frame_01_00120.png
Writing: frame_01_00180.png
Writing: frame_01_00240.png
Writing: frame_01_00300.png
Writing: frame_01_00360.png
Writing: frame_01_00420.png
Writing: frame_01_00480.png
Writing: frame_01_00540.png
Writing: frame_01_00600.png
Writing: frame_01_00660.png
Writing: frame_01_00720.png
Writing: frame_01_00780.png
Writing: frame_01_00840.png
Writing: frame_01_00900.png
Writing: frame_01_00960.png
Writing: frame_01_01020.png
Writing: frame_01_01080.png
Writing: frame_01_01140.png
Writing: frame_01_01200.png
Writing: frame_01_01260.png
Writing: frame_01_01320.png
Writing: frame_01_01380.png
Writing: frame_01_01440.png
Writing: frame_01_01500.png
Writing: frame_01_01560.png
Writing: frame_01_01620.png
Writing: frame_01_01680.png
Writing: frame_01_01740.png
Writing: frame_01_01800.png
Writing: frame_01_01860.png
Writing: frame_01_01920.png
Writing: frame_01_01980.png
Writing: frame_01_02040.png
Writing: frame_01_02100.png
Writing: frame_01_02

In [10]:
sample_image_in  = os.path.join(sample_dir, 'frame_01_02160.png')
sample_image_out = os.path.join(sample_dir, 'corrected_frame_01_02160.png')

In [12]:
mtx, dist = load_coefficients(config_file)
original = cv2.imread(sample_image_in)

dst = cv2.undistort(original, mtx, dist, None, None)

cv2.imwrite(sample_image_out, dst)

True